In [1]:
%cd /opt

/opt


In [2]:
%%capture
!tar xvf /kaggle/input/extract-prebuilt-kaldi-from-docker/kaldi.tar

In [3]:
%cd kaldi/egs

/opt/kaldi/egs


In [4]:
!git clone https://github.com/danijel3/ClarinStudioKaldi

Cloning into 'ClarinStudioKaldi'...
remote: Enumerating objects: 778, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 778 (delta 0), reused 0 (delta 0), pack-reused 775
Receiving objects: 100% (778/778), 35.26 MiB | 19.12 MiB/s, done.
Resolving deltas: 100% (262/262), done.


In [5]:
%cd ClarinStudioKaldi

/opt/kaldi/egs/ClarinStudioKaldi


In [6]:
%%capture
!conda install -c bioconda perl-perlio-gzip -y

In [7]:
import os
os.environ['LD_LIBRARY_PATH'] = '/opt/conda/lib:/opt/kaldi/tools/openfst-1.6.7/lib:/opt/kaldi/src/lib'

In [8]:
!cat path.sh|sed -e 's/~\/apps/\/opt/' > tmp
!mv tmp path.sh

In [9]:
!echo > local_clarin/clarin_pl_clean.sh

In [10]:
!ln -s ../wsj/s5/steps
!ln -s ../wsj/s5/conf
!ln -s ../wsj/s5/local
!ln -s ../wsj/s5/utils

In [11]:
!cp -r /kaggle/input/kaldi-clarinstudio-polish-decode-gmm/data /kaggle/working/
!cp -r /kaggle/input/kaldi-clarinstudio-polish-decode-gmm/exp /kaggle/working/

In [12]:
!ln -s /kaggle/working/exp
!ln -s /kaggle/working/data

In [13]:
!cp exp/tri3b/35.mdl exp/tri3b/final.mdl
!cp exp/tri3b/35.occs exp/tri3b/final.occs

In [14]:
%%writefile run.sh
. path.sh

export nj=40 ##number of concurrent processes
export nj_test=30 ## number of concurrent processes for test has to be <=30

#Performing tests on all the above systems:

./local/nnet3/run_ivector_common.sh --nj $nj_test --train_set train --test_sets test --gmm tri3b

./local_clarin/clarin_tdnn.sh
steps/nnet3/decode.sh --nj $nj_test --num-threads 4 --online-ivector-dir exp/nnet3/ivectors_test_hires \
          exp/tri3b/graph data/test_hires exp/nnet3/tdnn1a_sp/decode
./steps/oracle_wer.sh data/test_hires data/lang exp/nnet3/tdnn1a_sp/decode
./steps/lmrescore_const_arpa.sh data/lang_test data/lang_carpa data/test_hires exp/nnet3/tdnn1a_sp/decode exp/nnet3/tdnn1a_sp/decode_rs

# Same as above but using the chain framework - trains about the same, works much faster as has lower WER
./local_clarin/clarin_chain_tdnn.sh
./steps/oracle_wer.sh data/test_hires data/lang exp/chain/tdnn1f_sp/decode
./steps/lmrescore_const_arpa.sh data/lang_test data/lang_carpa data/test_hires exp/chain/tdnn1f_sp/decode exp/chain/tdnn1f_sp/decode_rs

# Getting results
find exp -name best_wer | while read f ; do cat $f ; done | sort -k2nr
find exp -name oracle_wer | while read f ; do echo -n "$f: " ; cat $f ; done | sort -k2nr

Overwriting run.sh


In [15]:
!bash run.sh

./local/nnet3/run_ivector_common.sh: preparing directory for low-resolution speed-perturbed data (for alignment)
utils/data/perturb_data_dir_speed_3way.sh: making sure the utt2dur and the reco2dur files are present
... in data/train, because obtaining it after speed-perturbing
... would be very slow, and you might need them.
utils/data/get_utt2dur.sh: data/train/utt2dur already exists with the expected length.  We won't recompute it.
utils/data/get_reco2dur.sh: data/train/wav.scp indexed by utt-id; copying utt2dur to reco2dur
sox: command not found
./local_clarin/clarin_tdnn.sh: expected file data/train_sp_hires/feats.scp to exist
steps/nnet3/decode.sh --nj 30 --num-threads 4 --online-ivector-dir exp/nnet3/ivectors_test_hires exp/tri3b/graph data/test_hires exp/nnet3/tdnn1a_sp/decode
steps/nnet2/check_ivectors_compatible.sh: WARNING: The directories do not contain iVector ID.
steps/nnet2/check_ivectors_compatible.sh: WARNING: That means it's you who's reponsible for keeping 
steps/nnet